In [1]:

import torch 
import torch.nn as nn 
import torchvision.transforms as transforms 
from torch.utils.data import DataLoader ,SubsetRandomSampler
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
import os
from skimage import io
import numpy as np
from PIL import Image
import torch.optim as optim
import torch.nn.functional as F


C:\Users\Moham\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:

class CarPaleteDataset(Dataset):
    globalIt = 0
    dic = {}
    def __init__(self , csv_file , root_dir , transform = None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir 
        self.transform = transform
        self.freq = 0
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir , self.df.iloc[index , 0])
        image = Image.open(img_path)
        label = self.df.iloc[index , 1]
        if label in CarPaleteDataset.dic.keys():
            self.freq += 1
        else:
            CarPaleteDataset.dic[label] = CarPaleteDataset.globalIt    
            CarPaleteDataset.globalIt += 1
            
        y_label = torch.tensor(CarPaleteDataset.dic[label])
        if self.transform:
            image = self.transform(image)
        return (image, y_label)


In [3]:


transformed = transforms.Compose([
    transforms.Resize(size = (128,128)) , 
    transforms.Grayscale(),
    transforms.ToTensor()
])
train_data = CarPaleteDataset(csv_file=r'C:\Users\Moham\Desktop\dataset\selected\train.csv' , root_dir=r"C:\Users\Moham\Desktop\dataset\selected\train" , 
                            transform=transformed)
test_data = CarPaleteDataset(csv_file=r'C:\Users\Moham\Desktop\dataset\selected\val.csv',root_dir=r"C:\Users\Moham\Desktop\dataset\selected\val" , 
                            transform=transformed)

samplerInd = np.random.choice(len(train_data) , 1000 , replace=False)
sampler = SubsetRandomSampler(samplerInd)

train_loader = DataLoader(dataset = train_data , batch_size=64 , shuffle = True)
test_loader = DataLoader(dataset = train_data , batch_size=64  , sampler=sampler)

ss = 0
for X,y in train_loader:
    ss += y.shape[0]
for X,y in test_loader:
    pass
#print(len(train_data.dic.keys()) , ss  , train_data.freq)

def getInfo():
    return train_loader , test_loader 

In [4]:


class CNN(nn.Module):
    def __init__(self , input_shape , hidden_units , output_shape):
        super().__init__()
        self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_channels= input_shape , out_channels= hidden_units,kernel_size=3 , stride=1 , padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels= hidden_units , out_channels= hidden_units,kernel_size=3 , stride=1 , padding=1),
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size=2 , stride=2)
        )
        
        self.conv_block2 = nn.Sequential(
        nn.Conv2d(in_channels= hidden_units , out_channels= hidden_units,kernel_size=3 , stride=1 , padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels= hidden_units , out_channels= hidden_units,kernel_size=3 , stride=1 , padding=1),
        nn.ReLU(), 
        nn.MaxPool2d(kernel_size=2 , stride=2)
        )
        
        self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*32*32 , out_features=output_shape)
        )
        
    def forward(self , x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.classifier(x)
        return x
    
model = CNN(input_shape= 1 , hidden_units=10 , output_shape=2152)
train_loader , test_loader = getInfo()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters() , lr = 1e-4)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def trainingStep(model , train_loader , criterion , optimizer , device):
    
    model.train()
    train_loss , correct , samples = 0 , 0 , 0
    
    for x , y in train_loader:
        x , y = x.to(device) , y.to(device)
        yPred = model(x)
        
        loss = criterion(yPred , y)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        correct += (y == yPred.argmax(dim = 1)).sum()
        samples += len(y)
        
    train_loss /= len(train_loader)
    acc = round(float(correct/samples)*100,3)
    
    return train_loss , acc

def testingStep(model , test_loader , criterion , device):
    
    test_loss , correct , samples = 0 , 0 , 0
    model.eval()
    with torch.inference_mode():
        
        for x , y in test_loader:

            x , y = x.to(device) , y.to(device)
            yPred = model(x)

            loss = criterion(yPred , y)
            test_loss += loss.item()
            
            correct += (y==yPred.argmax(dim = 1)).sum()
            samples += len(y)
    
    test_loss /= len(test_loader)
    acc = round(float(correct/samples)*100,3)
    
    return test_loss , acc

trainingLosses , trainingAcc = [] , []
testLosses , testAcc = [] , []

for epoch in range(6):
    
    train_loss , train_acc = trainingStep(model , train_loader , criterion , optimizer , device)
    test_loss , test_acc = testingStep(model , test_loader , criterion , device)
    
    trainingLosses.append(train_loss)
    trainingAcc.append(train_acc)
    testLosses.append(test_loss)
    testAcc.append(test_acc)
     
    print(f'trainingLoss= {train_loss} ,trainingAcc= {train_acc} |testLoss= {test_loss} ,testAcc= {test_acc}')

trainingLoss= 7.664087897691971 ,trainingAcc= 0.161 |testLoss= 7.545524209737778 ,testAcc= 0.3
trainingLoss= 7.495320955912272 ,trainingAcc= 0.372 |testLoss= 7.304140418767929 ,testAcc= 0.7
trainingLoss= 6.881800645436996 ,trainingAcc= 2.765 |testLoss= 5.314274102449417 ,testAcc= 18.4
trainingLoss= 3.3074609683110165 ,trainingAcc= 44.476 |testLoss= 1.080564547330141 ,testAcc= 79.8
trainingLoss= 0.8270000106631181 ,trainingAcc= 83.322 |testLoss= 0.30629503913223743 ,testAcc= 92.7
trainingLoss= 0.2807389352088555 ,trainingAcc= 93.475 |testLoss= 0.13356042187660933 ,testAcc= 96.7
